# SkyScan MQTT Config & Control

Most aspects of SkyScan can be configured and controlled using MQTT. This notebook walks through a number of different actions you can take to manage your deployment.

### Install Dependencies

The Cell below installs the dependencies required for this Notebook.

In [ ]:
!pip3 install paho-mqtt==1.6.1 coloredlogs
!wget https://raw.githubusercontent.com/IQTLabs/edgetech-core/main/core/base_mqtt_pub_sub.py

### MQTT Helper Functions

The following Cell has a number of functions that are used to format and send MQTT messages.

In [ ]:
from base_mqtt_pub_sub import BaseMQTTPubSub
from typing import Any, Dict
from time import sleep
import os
from datetime import datetime
import json
import logging

class MqttHelper(BaseMQTTPubSub):
    def __init__(
        self: Any,
        publish_topic: str,
        orientation_topic: str,
        config_topic: str,
        object_detection_topic: str,
        manual_control_topic: str,
        ledger_topic: str,
        debug: bool = False,
        **kwargs: Any,
    ):
        super().__init__(**kwargs)
        self.orientation_topic = orientation_topic
        self.publish_topic = publish_topic
        self.config_topic = config_topic
        self.object_detection_topic = object_detection_topic
        self.manual_control_topic = manual_control_topic
        self.ledger_topic = ledger_topic
        self.debug = debug

        self.connect_client()
        sleep(1)
        self.publish_registration("Manual Sender Registration")

        print(f"Config Topic: {self.config_topic}")
        print(f"Orientation Topic: {self.orientation_topic}")
        print(f"Publish Topic: {self.publish_topic}")
        print(f"Object Detection Topic: {self.object_detection_topic}")
        print(f"Manual Control Topic: {self.manual_control_topic}")
        print(f"Ledger Topic: {self.ledger_topic}")
        
    # Used to target a static object at a specific Lat, Lon, Alt
    def sendCoordinates(self, name, lat,lon, alt):
        target = {
            "timestamp": "2023-01-01-00-00-00",
            "data": {
                "object_id": name,
                "object_type": "aircraft",
                "timestamp": 1.0,
                "latitude": float(lat),
                "longitude": float(lon),
                "altitude": float(alt),
                "track": 0.0,
                "horizontal_velocity": 0.0,
                "vertical_velocity": 0.0
            }
        }
        
        payload_json = self.generate_payload_json(
            push_timestamp=int(datetime.utcnow().timestamp()),
            device_type="TBC",
            id_="TBC",
            deployment_id="TBC",
            current_location="TBC",
            status="Debug",
            message_type="Event",
            model_version="null",
            firmware_version="v0.0.0",
            data_payload_type="Selected Object",
            data_payload=json.dumps(target["data"]),
        )
        self.publish_to_topic(self.publish_topic, payload_json)    


    # Used to track a specific aircraft by its ICAO24           
    def sendObjectIDOverride(self, object_id):
        print(f"Sending Object ID Override - {object_id}")
        object_id_msg = self.generate_payload_json(
            push_timestamp=int(datetime.utcnow().timestamp()),
            device_type=os.environ.get("DEVICE_TYPE", "Detector"),
            id_="Mission Control",
            deployment_id=os.environ.get(
                "DEPLOYMENT_ID", f"Unknown-Location"
            ),
            current_location=os.environ.get("CURRENT_LOCATION", "-90, -180"),
            status="Debug",
            message_type="Event",
            model_version="null",
            firmware_version="v0.0.0",
            data_payload_type="ObjectIDOverride",
            data_payload=object_id,
        )
        # Send JSON message over MQTT
        self.publish_to_topic(self.ledger_topic, object_id_msg)
        print(f"Sent Object ID Override: {object_id_msg}")


    # Update the orientation correction for the camera
    def sendOrientation(self, yaw, pitch, roll):
        print(f"Sending Orientation - Yaw: {yaw}, Pitch: {pitch}, Roll: {roll}")
        orientation_msg = self.generate_payload_json(
            push_timestamp=int(datetime.utcnow().timestamp()),
            device_type=("Collector"),
            id_="Mission Control",
            deployment_id="TBC",
            current_location="-90, -180",
            status="Debug",
            message_type="Event",
            model_version="null",
            firmware_version="v0.0.0",
            data_payload_type="Orientation",
            data_payload=json.dumps(
                {
                "tripod_yaw": yaw,
                "tripod_pitch": pitch,
                "tripod_roll": roll,
                }
            ),
        )
        print(f"Sending  Orientation: {orientation_msg}")
        self.publish_to_topic(self.orientation_topic, orientation_msg)

    # Send configuration messages for the PTZ controller
    def sendConfig(self, json_config: Dict[str, Any]):
        config_msg = self.generate_payload_json(
            push_timestamp=int(datetime.utcnow().timestamp()),
            device_type=("Collector"),
            id_="Mission Control",
            deployment_id="TBC",
            current_location="-90, -180",
            status="Debug",
            message_type="Event",
            model_version="null",
            firmware_version="v0.0.0",
            data_payload_type="Configuration",
            data_payload=json.dumps(json_config),
        )
        print(f"Sending Config: {config_msg}")
        self.publish_to_topic(self.config_topic, config_msg)
        
    # Manually control the PTZ camera
    def sendManualControl(self, json_config: Dict[str, Any]):
    
        config_msg = self.generate_payload_json(
            push_timestamp=int(datetime.utcnow().timestamp()),
            device_type=("Collector"),
            id_="Mission Control",
            deployment_id="TBC",
            current_location="-90, -180",
            status="Debug",
            message_type="Event",
            model_version="null",
            firmware_version="v0.0.0",
            data_payload_type="Manual Control",
            data_payload=json.dumps(json_config),
        )
        print(f"Sending Manual Control: {config_msg}")
        self.publish_to_topic(self.manual_control_topic, config_msg)        

    def main(self: Any) -> None:
        print('this is a placeholder and does nothing')

## Create Instance

Create an instance of the MQTT helper in order to send messages. Update the following variables to reflect what is used in the SkyScan deployment.

In [ ]:
project_name = "skyscan"
hostname = "dev_laptop"
mqtt_ip='127.0.0.1'

Now, create an instance of the helper class. You only need to update the MQTT topics if you have made changes in the `.env` file.

In [ ]:
mqtt_helper = MqttHelper(mqtt_ip=mqtt_ip,
    orientation_topic=f"/{project_name}/{hostname}/Orientation/edgetech-auto-orienter/JSON",
    publish_topic=f"/{project_name}/{hostname}/Object/skyscan-c2/JSON",
    config_topic=f"/{project_name}/{hostname}/Config/skyscan-c2/JSON",
    object_detection_topic=f"/{project_name}/{hostname}/Detections/edgetech-yolo-detect/JSON",
    manual_control_topic=f"/{project_name}/{hostname}/Manual_Control/edgetech-axis-ptz-controller/JSON",
    ledger_topic=f"/{project_name}/{hostname}/Ledger/edgetech-object-ledger/JSON",
)

### Update Orientation
You can manually set the Yaw, Pitch, and Roll corrections for the orientation of the camera. Use this to fine tune the pointing of the camera and correct for it not being level.

In [ ]:
# update these values to explore orientation corrections
yaw = 204.18
pitch = -2.76  #-0.46
roll =  -2.75 #-2.0

mqtt_helper.sendOrientation(yaw, pitch, roll) 

### Autofocus Control

Just leave this set to True

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"auto_focus": True}})

### Loop Interval

This is how quickly the plane tracking control loop inside the Axis PTZ Controller runs, measured in seconds

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"tracking_interval": 0.25}})

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"lead_time": 0.0}})

### Camera Location

Adjust the cameras location and altitude. 

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"tripod_latitude": 38.925747053571556, "tripod_longitude": -77.05186490913148, "tripod_altitude": 21}})


### Pan / Tilt Gain

The Pan / Tilt Gain is used in the control system in the Axis PTZ Controller module. It the camera starts oscillating rapidly back and forth, try lowering the gain. If the camera is having trouble keeping up with the plane, try increasing it.

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"pan_gain": 10}})
mqtt_helper.sendConfig({"axis-ptz-controller":{"tilt_gain": 10}})

You can additionally use the rate of change of the aircraft's angular velocity as a gain element in the control loop. It can help when aircraft are flying directly overhead.

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"pan_derivative_gain_max": 1}})  #4
mqtt_helper.sendConfig({"axis-ptz-controller":{"tilt_derivative_gain_max": 1}})  #2

These are the max pan/tilt rates for the camera. You generally should not need to mess with these after they have been set.

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"pan_rate_max": 350}})
mqtt_helper.sendConfig({"axis-ptz-controller":{"tilt_rate_max": 350}})

### Camera Zoom

Set the zoom for the camera. It should be between 0 (wide) - 9999 (fully zoomed in)

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"zoom":9999}})

In [ ]:
mqtt_helper.sendConfig({"axis-ptz-controller":{"lead_time":0.0 }})

### Min Altitude

The minimum altitude for an aircraft to be tracked. This is helpful if you want to track only cruising level aircraft. This is measured in meters.

In [ ]:
mqtt_helper.sendConfig({"skyscan-c2":{"min_altitude": 0}})

### Max Altitude

The maximum altitude for an aircraft to be tracked. This is helpful when there is cloud cover and you want to ignore aircraft that are above it. This is measured in meters.

In [ ]:
mqtt_helper.sendConfig({"skyscan-c2":{"max_altitude": 3500}})

### Min Tilt
If there are obstructions around the camera, you can use this setting to ignore all aircraft below the specified tilt value.

In [ ]:
mqtt_helper.sendConfig({"skyscan-c2":{"min_tilt": 0}})

### Target Specific Aircraft

Use the ICAO24 code for the aircraft. Keep all of the letters lowercase. Send an empty string to reset back to automatic tracking.

In [ ]:
mqtt_helper.sendObjectIDOverride("")

### Target Specific Static Coordinates
Let's you move the camera to point at a specific static target. This can be helpful for leveling the camera. 

In [ ]:
mqtt_helper.sendCoordinates("Washington Monument", 38.889444, -77.035278, 139.0)

In [ ]:
mqtt_helper.sendCoordinates("Washington National Cathederal", 38.930556, -77.070833, 139.0)

# Manual Control
The Camera's motion can also be manually controlled. You can specify Pan/Tilt values for the camera to move to and also Pan/Tilt rates for the camera to move at.

In [ ]:
mqtt_helper.sendManualControl({"pan": 0, "tilt": 0})
sleep(5)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": 2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": -2.00, "tilt_rate": -2.0})
sleep(15)
mqtt_helper.sendManualControl({"pan_rate": 2.00, "tilt_rate": -2.0})
sleep(15)